In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta, date, time
import pickle
from progressbar import ProgressBar

In [41]:
#pip install progressbar
pbar = ProgressBar()

In [268]:
#df_2017 = pd.read_csv('C:/Users/Marc/git/BananaCode/data/LosAngeles/la_2017.csv')
df_2018 = pd.read_csv('C:/Users/Marc/git/BananaCode/data/LosAngeles/la_2018.csv')  
#df_2019 = pd.read_csv('C:/Users/Marc/git/BananaCode/data/LosAngeles/la_2019.csv')  
df_weather = pd.read_csv('C:/Users/Marc/git/BananaCode/data/LosAngeles/weather_hourly_la.csv')  
#df_2018.head(10)

**Only one trip at the same time**

In [246]:
df_id = df_2018.sort_values(by=['bike_id','start_time'], ascending=True)

temp = df_id[['start_time','end_time', 'bike_id']]
temp['next_ride_start']= temp['start_time'].shift(periods=-1)
temp['next_bike_id']= temp['bike_id'].shift(periods=-1)
temp['id_compare'] = 0
temp['check'] = 0
temp = temp[:1000] #only first 10000

dflength = len(temp)
count=0
for index, row in temp.iterrows():
    count=count+1
    print(str(round(((count/dflength)*100),4))+'%', end="\r")
    if(row['bike_id'] == row['next_bike_id']):
        temp.loc[index,'id_compare'] = True
        if((row['next_ride_start'] < row['end_time'])):
            temp.loc[index,'check'] = False
        else:
            temp.loc[index,'check'] = True             
    else:
        temp.loc[index,'id_compare'] = False
temp[(temp['check']==False)&(temp['id_compare']==True)]

#pickle write ATTENTION: DONT'T UNCOMMENT!
#filename = 'trips_same_time'
#outfile = open(filename,'wb')
#pickle.dump(temp,outfile)
#outfile.close()

<ipython-input-246-ac84ea55e38a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['next_ride_start']= temp['start_time'].shift(periods=-1)
<ipython-input-246-ac84ea55e38a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['next_bike_id']= temp['bike_id'].shift(periods=-1)
<ipython-input-246-ac84ea55e38a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

,start_time,end_time,bike_id,next_ride_start,next_bike_id,id_compare,check
118456,2018-06-03 12:34:00,2018-06-03 12:52:00,4728,2018-06-03 12:34:00,4728.0,True,False
118545,2018-06-03 13:52:00,2018-06-03 16:09:00,4728,2018-06-03 13:52:00,4728.0,True,False


In [261]:
#pickle read in
infile = open(filename,'rb')
df_trips_same_time = pickle.load(infile)
infile.close()

**Find wrong entrys in dataset (!contains double entrys!)**

In [263]:
df_trips_same_time[(df_trips_same_time['check']==False)&(df_trips_same_time['id_compare']==True)]

,start_time,end_time,bike_id,next_ride_start,next_bike_id,id_compare,check
118456,2018-06-03 12:34:00,2018-06-03 12:52:00,4728,2018-06-03 12:34:00,4728.0,True,False
118545,2018-06-03 13:52:00,2018-06-03 16:09:00,4728,2018-06-03 13:52:00,4728.0,True,False
193634,2018-08-19 14:32:00,2018-08-19 20:32:00,5006,2018-08-19 14:32:00,5006.0,True,False
193636,2018-08-19 14:32:00,2018-08-19 14:52:00,5006,2018-08-19 14:32:00,5006.0,True,False
194167,2018-08-19 20:32:00,2018-08-20 02:32:00,5006,2018-08-19 20:32:00,5006.0,True,False
...,...,...,...,...,...,...,...
305070,2018-12-20 18:14:00,2018-12-20 18:29:00,15047,2018-12-20 18:28:00,15047.0,True,False
307029,2018-12-24 08:43:00,2018-12-24 09:13:00,15047,2018-12-24 08:58:00,15047.0,True,False
287878,2018-11-26 17:39:00,2018-11-26 18:09:00,15055,2018-11-26 17:45:00,15055.0,True,False
308086,2018-12-26 09:44:00,2018-12-26 10:04:00,15070,2018-12-26 10:03:00,15070.0,True,False


**Find double entries to ignore them when searching for wrong entrys**

In [248]:
df_found = df_trips_same_time[(df_trips_same_time['check']==False)&(df_trips_same_time['id_compare']==True)]
df_found['double'] = 0

duplicates = df_2018.duplicated(keep=False).reset_index()
#duplicates = duplicates[duplicates[0]==True]
print('Duplicates found: '+str(len(duplicates[duplicates[0]==True])/2))

for index, row in df_found.iterrows():
    if (duplicates.iloc[index,1]==True):
        df_found.loc[index,'double'] = True
    else:
        df_found.loc[index,'double'] = False
    

print('Marked as duplicates in df_found: '+str(len(df_found[df_found['double']== True])))

#print out the cleaned (double entry) wrong entrys
df_found[df_found['double']== False]


<ipython-input-248-916da822ace0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_found['double'] = 0


Duplicates found: 42.0
Marked as duplicates in df_found: 44


C:\Users\Marc\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,start_time,end_time,bike_id,next_ride_start,next_bike_id,id_compare,check,double
193634,2018-08-19 14:32:00,2018-08-19 20:32:00,5006,2018-08-19 14:32:00,5006.0,True,False,False
193636,2018-08-19 14:32:00,2018-08-19 14:52:00,5006,2018-08-19 14:32:00,5006.0,True,False,False
194167,2018-08-19 20:32:00,2018-08-20 02:32:00,5006,2018-08-19 20:32:00,5006.0,True,False,False
191462,2018-08-17 16:16:00,2018-08-17 22:16:00,5716,2018-08-17 16:16:00,5716.0,True,False,False
193112,2018-08-18 23:01:00,2018-08-19 05:01:00,5720,2018-08-18 23:01:00,5720.0,True,False,False
...,...,...,...,...,...,...,...,...
305070,2018-12-20 18:14:00,2018-12-20 18:29:00,15047,2018-12-20 18:28:00,15047.0,True,False,False
307029,2018-12-24 08:43:00,2018-12-24 09:13:00,15047,2018-12-24 08:58:00,15047.0,True,False,False
287878,2018-11-26 17:39:00,2018-11-26 18:09:00,15055,2018-11-26 17:45:00,15055.0,True,False,False
308086,2018-12-26 09:44:00,2018-12-26 10:04:00,15070,2018-12-26 10:03:00,15070.0,True,False,False


**Exapmle: Wrong entrys: 193634, 193636, 194167**

In [271]:
#df_trips_same_time dataframe
df_trips_same_time[(df_trips_same_time['bike_id']==5006)&(df_trips_same_time['start_time']>='2018-08-19 14:32:00')].head(10)

,start_time,end_time,bike_id,next_ride_start,next_bike_id,id_compare,check
193634,2018-08-19 14:32:00,2018-08-19 20:32:00,5006,2018-08-19 14:32:00,5006.0,True,False
193636,2018-08-19 14:32:00,2018-08-19 14:52:00,5006,2018-08-19 14:32:00,5006.0,True,False
193637,2018-08-19 14:32:00,2018-08-19 14:52:00,5006,2018-08-19 20:32:00,5006.0,True,True
194167,2018-08-19 20:32:00,2018-08-20 02:32:00,5006,2018-08-19 20:32:00,5006.0,True,False
194168,2018-08-19 20:32:00,2018-08-19 20:52:00,5006,2018-08-21 15:26:00,5006.0,True,True
195828,2018-08-21 15:26:00,2018-08-21 15:58:00,5006,2018-08-24 20:42:00,5006.0,True,True
199253,2018-08-24 20:42:00,2018-08-24 21:53:00,5006,2018-08-24 22:12:00,5006.0,True,True
199301,2018-08-24 22:12:00,2018-08-24 22:33:00,5006,2018-08-25 14:06:00,5006.0,True,True
199754,2018-08-25 14:06:00,2018-08-25 14:15:00,5006,2018-08-25 16:11:00,5006.0,True,True
199945,2018-08-25 16:11:00,2018-08-25 16:29:00,5006,2018-08-26 14:53:00,5006.0,True,True


In [272]:
#original dataframe
df_id[(df_id['bike_id']==5006)&(df_id['start_time']>='2018-08-19 14:32:00')].head(10)

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name
193634,2018-08-19 14:32:00,2018-08-19 20:32:00,4126,4126,5006,Monthly Pass,Cabrillo Beach,Cabrillo Beach
193636,2018-08-19 14:32:00,2018-08-19 14:52:00,4126,4131,5006,Monthly Pass,Cabrillo Beach,Downtown Harbor
193637,2018-08-19 14:32:00,2018-08-19 14:52:00,4126,4126,5006,Monthly Pass,Cabrillo Beach,Cabrillo Beach
194167,2018-08-19 20:32:00,2018-08-20 02:32:00,4126,4126,5006,Monthly Pass,Cabrillo Beach,Cabrillo Beach
194168,2018-08-19 20:32:00,2018-08-19 20:52:00,4126,4131,5006,Monthly Pass,Cabrillo Beach,Downtown Harbor
195828,2018-08-21 15:26:00,2018-08-21 15:58:00,4134,4131,5006,Monthly Pass,USS IOWA,Downtown Harbor
199253,2018-08-24 20:42:00,2018-08-24 21:53:00,4131,4132,5006,Walk-up,Downtown Harbor,Fanfare Fountain
199301,2018-08-24 22:12:00,2018-08-24 22:33:00,4132,4131,5006,Walk-up,Fanfare Fountain,Downtown Harbor
199754,2018-08-25 14:06:00,2018-08-25 14:15:00,4131,4133,5006,Walk-up,Downtown Harbor,Ports O'Call
199945,2018-08-25 16:11:00,2018-08-25 16:29:00,4133,4132,5006,Walk-up,Ports O'Call,Fanfare Fountain


# Notizen

In [ ]:
#pickle write
filename = 'dogs'
outfile = open(filename,'wb')
pickle.dump(dogs_dict,outfile)
outfile.close()

In [ ]:
#pickle read in
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()

In [254]:
#Mona:
#doppelte daten --> gleiche reihen -->FUNKTION googlen
#außreiser Fahrzeit
#daten als datetime? Sekunden weg
#Marc:
#bike ID kann (zu einem Zeitpunkt) nur einmal unterwegs sein DONE
#Informationen zu den Spalten suchen: https://bikeshare.metro.net/about/data/DONE
#Stationen: Alles außerhalb raus PROBLEM: ENTFERNUNG
#Km/h Ausreißer? --> Station Entfernung??? PROBLEM: ENTFERNUNG
#Ercan:
#Start und Endtime gleich --> Raus
#Endzeitpunkt vor Startzeitpunkt -->raus
#Start Station ID muss lgeich Start Station Name sein -->End analog --> ID oder Name Falsch?